### 나이브 베이즈 구현

#### 밑바닥부터 구현하는 나이브 베이즈

In [22]:
# 간단한 데이터셋 정의
import numpy as np 
X_train = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [1, 1, 0],
    [1, 1, 0]
])
Y_train = ['Y', 'N', 'Y', 'Y']
X_test = np.array([[1, 1, 0]])

In [6]:
# 사전확률을 계산하기 위해 데이터를 레이블별로 그룹화하고 클렉스별로 인덱스를 기록
def get_label_indices(labels):
    """
    레이블과 반환 인덱스 기준으로 샘플 그룹화
    @para labels: 레이블 리스트
    @return label_indices: dictionary, {class1: [indices], class2: [indices]}
    """
    from collections import defaultdict
    # defaultdict를 사용하여 기본값으로 빈 리스트를 가지는 딕셔너리 생성
    label_indices= defaultdict(list)
    # 주어진 레이블 리스트를 순회하며 각 레이블의 인덱스를 기록
    for index, label in enumerate(labels):
        label_indices[label].append(index)
    
    # 완성된 딕셔너리 반환
    return label_indices

In [8]:
label_indices = get_label_indices(Y_train)
print('label_indices: \n',label_indices)

label_indices: 
 defaultdict(<class 'list'>, {'Y': [0, 2, 3], 'N': [1]})


In [11]:
#label_indices를 이용해 사전확률 계산 
def get_prior(label_indices):
    """
    훈련 샘플을 이용한 사전확률 계산
    @para label_indices: 클래스별로 그룹화된 샘플 인덱스
    @return prior: dictionary, key = 클래스 레이블, value = 사전확률
    """
    # 각 클래스의 샘플 수를 카운트하여 사전확률의 분자를 구함
    prior = {label: len(indices) for label, indices in label_indices.items()}
    # 전체 샘플 수를 계산하여 사전확률의 분모를 구함
    total_count = sum(prior.values())
    
    # 각 클래스의 사전확률을 계산하여 업데이트
    for label in prior:
        prior[label] /= total_count
    return prior

In [13]:
prior = get_prior(label_indices)
print("Prior:",prior)

Prior: {'Y': 0.75, 'N': 0.25}


In [19]:
#사전 확률을 통해 조건부 확률에 해당하는 우도 계산
def get_likelihood(features, label_indices, smoothing=0):
    """
    훈련 샘플을 이용한 우도 계산 
    @param features: 특징 행렬
    @param label_indices: 클래스별로 그룹화된 샘플 인덱스 
    @param smoothing: int, 가산(additive) 평활화 계수
    @return likelihood: dictionary, key = 클래스, value = 조건부 확률 P(feature | class)벡터
    """
    likelihood ={}
    for label, indices in label_indices.items():
        # 해당 클래스에 속하는 샘플들의 특징 값의 합을 계산
        likelihood[label] = features[indices, :].sum(axis=0) + smoothing
        total_count = len(indices) # 해당 클래스에 속하는 샘플의 수
        # 가산 평활화를 적용한 후, 각 특징에 대한 조건부 확률을 계산
        likelihood[label] = likelihood[label] / (total_count + 2*smoothing)
    return likelihood

In [23]:
#평활화값 1로 설정
smoothing =1
likelihood= get_likelihood(X_train, label_indices, smoothing)
print("Likelihood:\n",likelihood)

Likelihood:
 {'Y': array([0.6, 0.8, 0.4]), 'N': array([0.33333333, 0.33333333, 0.66666667])}


In [25]:
#테스트/새로운 샘플에 대한 사후확률 계산 
def get_posterior(X, prior, likelihood):
    """
    사전확률과 우도를 이용한 테스트 샘플의 사후확률 계산 
    @param X: 테스트 샘플
    @param prior: dictionary, key = 클래스 레이블, value = 사전확률
    @param likelihood: dictionary, key = 클래스 레이블. value = 조건부 확률 벡터
    @return posteriors: dictionary, key = 클래스 레이블 , value = 사후확률
    """
    posteriors =[]
    for x in X:
        #사후확률은 사전확률*우도에 비례한다. 
        
        # 사후확률을 계산하기 위해 사전확률 복사하여 시작
        posterior = prior.copy() 
        for label, likelihood_label in likelihood.items(): 
            for index, bool_value in enumerate(x):
                # 해당 특성이 참인 경우 우도를 곱하고, 거짓인 경우 1에서 우도를 뺀 값을 곱함
                posterior[label] *= likelihood_label[index] if bool_value else (1 - likelihood_label[index])
        #모든 합이 1이 되도록 정규화한다.
        sum_posterior = sum(posterior.values())
        for label in posterior:
            # 사후확률이 무한대로 계산된 경우 1.0으로 설정 (오버플로 방지)
            if posterior[label] == float('inf'):
                posterior[label] = 1.0
            else:
                 # 사후확률 정규화
                posterior[label] /= sum_posterior
        posteriors.append(posterior.copy())
    return posteriors
        

In [26]:
posterior = get_posterior(X_test, prior, likelihood)
print('Posterior:\n',posterior)

Posterior:
 [{'Y': 0.9588951002959553, 'N': 0.041104899704044726}]
